In [1]:
import numpy as np
import sympy as sp
import math
from sympy import symbols, integrate, pprint, solve
from numpy import pi, sqrt, rad2deg
import numpy as np
import sympy as sp

**!!! All the unit should be in [N] and [mm]**

In [2]:
def check_safe(d, factors, Se, Sut, Sy, M_a, M_m, T_a, T_m):
    """
    d: diameter
    factors: {'K_f': K_f, 'K_fs': K_fs, 'ka': ka, 'kb': kb, 'kc': kc, 'kd':kd, 'ke': ke}
    Se: endurance limit
    Sut, Sy: ultimate and yeilding strength of the material
    return: {'sigma_a_prime': sigma_a_prime, 'sigma_m_prime': sigma_m_prime, 
            'fatigue safety factor': n_f, 'yeilding safety factor': n_y}
    """
    K_f = factors['K_f']
    K_fs = factors['K_fs']
    ka = factors['ka']
    kb = factors['kb']
    kc = factors['kc']
    kd = factors['kd']
    ke = factors['ke']
    
    sigma_a = (32*K_f*M_a)/(np.pi*d**3) 
    tau_a = (16*K_fs*T_a)/(np.pi*d**3)
    sigma_m = (32*K_f*M_m)/(np.pi*d**3)
    tau_m = (16*K_fs*T_m)/(np.pi*d**3)

    sigma_a_prime = (sigma_a**2 + 3*tau_a**2)**(1/2)
    sigma_m_prime = (sigma_m**2 + 3*tau_m**2)**(1/2)
    
    n_f = ((sigma_a_prime/Se) + (sigma_m_prime/Sut))**(-1)
    n_y = Sy/(sigma_a_prime + sigma_m_prime)
    
    return {'sigma_a_prime': sigma_a_prime, 'sigma_m_prime': sigma_m_prime, 
            'fatigue safety factor': n_f, 'yeilding safety factor': n_y}

In [3]:
# unit:[mm]
AD = 30  # D is the mid of the wide of gear
AC = 35   # C is the mid of the wide of scissor
CB = 15 
BO = 30
AC = 45

### Froce Analysis in the xy plane

In [4]:
#数据可以手算，但不要用计算机算出来约分小数点，会有误差

T = 25 #Nmm  # From the motor
F = 50 #N   #From the scissor
F_Ax = -12.5 #N
F_Bx = -37.5 #N
V_A = F_Ax

In [5]:
# Moment functions
V_AC = V_A
M_A = V_A*(AD + DC)

V_B = F + F_Ax
V_CB = V_B

M_B = V_AC*CB - F_Ax*(AD+DC)

x = sp.symbols('x')
Mfunc_OA = (M_A/AC)*(x)
Mfunc_AB = ((M_B - M_A)/CB)*(x - AC) + M_A #Nmm

In [6]:
V_B #N

37.5

In [7]:
np.array([M_A, M_B])*1e-3  #Nm

array([-0.5625,  0.375 ])

### Let's first consider the force on A

In [29]:
M_a = 21264
M_m = 0
T_a = 0
T_m = 0
M_a, M_m, T_a, T_m  # Nmm

(21264, 0, 0, 0)

Stress Concentration (**Should be modified manually when the data change.**)

In [30]:
Kt = 2.1
Kts = 1.7
Kf = Kts
Kfs = Kts

Endurance limit (**Should be modified manually when the data change.**)

In [31]:
#Choose 1020 CD steel
Sut = 690 #MPa
Sy = 580 #MPa

# ka
a = 3.04
b = -0.217
ka = a*(Sut**b)
ka

0.7359415631796432

In [32]:
kb = 1.24*12**(-0.107)
kc, kd, ke = 1, 1, 1
kb

0.9504938076488614

In [33]:
Se_prime = 0.5*Sut  # [MPa]
Se = ka*kb*kc*kd*ke*Se_prime # [MPa]
Se_prime, Se  # [MPa]

(345.0, 241.3302250148176)

In [34]:
n = 1.5  #Assume minimum safety factor is 1.5

A = math.sqrt(4 * (Kf * M_a)**2 + 3 * (Kfs * T_a)**2)
B = math.sqrt(4 * (Kf * M_m)**2 + 3 * (Kfs * T_m)**2)
d_prime = (16*n*((A/Se)+(B/Sut))/np.pi)**(1/3)

print("A: ", A)
print("B: ", B)
print('d: ',d_prime)

A:  72297.59999999999
B:  0.0
d:  13.178253089157693


Based on the calculation above, select the diameters and recalculate all the factors. (**Should be modified manually when the data change.**)
Typical $D/d$ ratio for support at a shoulder: $D/d = 1.5$  \
Assume $r/d = 0.1$ for well rounded shoulder

In [35]:
d = 12 # [mm] Choose the standard value
D_prime = 1.5*d  # (D/d)*d
r = 0.1*d  # Fillet radius (r/d)*d
D_prime, r

(18.0, 1.2000000000000002)

In [36]:
D = 18 # Choose the standard value 

In [38]:
# From Fig.(A-15-9&8) and Fig.(6-26&27)
K_t = 1.7
K_ts = 1.4
q = 0.76
q_s = 0.8

In [42]:
K_f = 1+q*(K_t - 1)
K_fs = 1+q_s*(K_ts - 1)
ka = 0.8
kb = 1.24*(d)**(-0.107)

factors = {'K_f': K_f, 'K_fs': K_fs, 'ka': ka, 'kb': kb, 'kc': kc, 'kd':kd, 'ke': ke}
Se = ka*kb*kc*kd*ke*Se_prime
Se

262.33629091108577

In [43]:
factors

{'K_f': 1.532,
 'K_fs': 1.3199999999999998,
 'ka': 0.8,
 'kb': 0.9504938076488614,
 'kc': 1,
 'kd': 1,
 'ke': 1}

In [41]:
check_safe(d, factors, Se, Sut, Sy, M_a, M_m, T_a, T_m)

{'sigma_a_prime': 192.02602694726252,
 'sigma_m_prime': 0.0,
 'fatigue safety factor': 1.3661496573229266,
 'yeilding safety factor': 3.020423893680254}

### Let's consider the force on C

In [20]:
M_a = 6402.308
M_m = 0
T_a = 0
T_m = 25000
M_a, M_m, T_a, T_m  # Nmm

(6402.308, 0, 0, 25000)

In [21]:
r = 1.2 #mm
D = 18 #mm
d = 12 #mm
D/d

1.5

In [22]:
# From Fig.(A-15-9&8) and Fig.(6-26&27)
K_t = 2.1
K_ts = 1.7
q = 0.69
q_s = 0.71

In [23]:
K_f = 1+q*(K_t - 1)
K_fs = 1+q_s*(K_ts - 1)
ka = 0.8
kb = 1.24*(d)**(-0.107)

factors = {'K_f': K_f, 'K_fs': K_fs, 'ka': ka, 'kb': kb, 'kc': kc, 'kd':kd, 'ke': ke}
Se = ka*kb*kc*kd*ke*Se_prime
Se

262.33629091108577

In [24]:
factors

{'K_f': 1.759,
 'K_fs': 1.4969999999999999,
 'ka': 0.8,
 'kb': 0.9504938076488614,
 'kc': 1,
 'kd': 1,
 'ke': 1}

In [25]:
check_safe(d, factors, Se, Sut, Sy, M_a, M_m, T_a, T_m)

{'sigma_a_prime': 66.38328963455359,
 'sigma_m_prime': 191.05077695519046,
 'fatigue safety factor': 1.8870355606481553,
 'yeilding safety factor': 2.253004070841597}

In [26]:
3.04*(Sut)**(-0.217)

0.7359415631796432

### Deflection

In [27]:
def deflection_slope(Mz1, Mz2, E, d1, d2, OA, OB):
    """
    Mz1, Mz2, Mz3:  Nmm
    AE, AH, AB:  mm
    E: MPa
    d1:  mm
    d2:  mm
    return: deflection and slope for each segment
    """
    I1 = pi*d1**4/64
    I2 = pi*d2**4/64
    C1, C2, C3, C4, C5, C6 = symbols('C1, C2, C3, C4, C5, C6',real=True)
    EIdv1dx = integrate(Mz1,x) + C1
    EIv1 = integrate(EIdv1dx,x) + C2

    EIdv2dx = integrate(Mz2,x) + C3
    EIv2 = integrate(EIdv2dx,x) + C4

    eq1 = EIv1.subs(x,0)/E/I1
    eq2 = EIdv1dx.subs(x,OA)/E/I1 - EIdv2dx.subs(x,OA)/E/I2
    eq3 = EIv1.subs(x,OA)/E/I1 
    eq4 = EIv2.subs(x,OB)/E/I2
    
    c1t6 = solve((eq1,eq2,eq3,eq4))
    
    v1 = EIv1.subs(c1t6)/E/I1
    v2 = EIv2.subs(c1t6)/E/I2
    
    dv1dx = EIdv1dx.subs(c1t6)/E/I1
    dv2dx = EIdv2dx.subs(c1t6)/E/I2
    
    defl = {"v1": v1, "v2": v2}
    slop = {"s1": dv1dx, "s2": dv2dx}
    
    return defl, slop

In [28]:
d1 = 300 #mm
d2 = 250 #mm
E = 205.0e3 # N/mm/mm  typical value for steel
Mz1 = Mfunc_OA
Mz2 = Mfunc_AB
# check
Mz1.subs({x:OA}), Mz2.subs({x:OA})

NameError: name 'OA' is not defined

In [ ]:
defl, slop = deflection_slope(Mz1, Mz2, E, d1, d2, OA, OB)

In [ ]:
v1, v2 = defl['v1'], defl['v2']
s1, s2 = slop['s1'], slop['s2']
print('v1:', v1)
print('v2:', v2)

In [ ]:
print('s1:', s1)
print('s2:', s2)

### Checking deflections and slopes

#### At bearing A:

In [ ]:
rotzA = float(s1.subs(x,OA))
rotyA = 0

slA = sqrt(rotzA*rotzA + rotyA*rotyA)
slA #rad

#### At bearing B:

In [ ]:
rotzB = float(s2.subs(x,OB))
rotyB = 0

slB = sqrt(rotzB*rotzB + rotyB*rotyB)
slB #rad

#### At O:

In [ ]:
rotzO = float(s2.subs(x,0))
rotyO = 0

slO = sqrt(rotzO*rotzO + rotyO*rotyO)
slO #rad